<a href="https://colab.research.google.com/github/Muppidathe/ml_project/blob/master/ann_pytorch(classification_with_cross_val).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np

In [41]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Churn_Modelling.csv')
df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# data transformation

In [42]:
#drop unwanted column
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
from sklearn.preprocessing import LabelEncoder
print(df['Gender'].unique())
gender_encoder=LabelEncoder()
df['Gender']=gender_encoder.fit_transform(df['Gender'])
df.head()

['Female' 'Male']


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [44]:
from sklearn.preprocessing import OneHotEncoder
print(df['Geography'].unique())
print('len: ',len(df['Geography'].unique()))
geo_encoder=OneHotEncoder()
temp_df=geo_encoder.fit_transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp_df,columns=geo_encoder.get_feature_names_out())
temp_df.head()

['France' 'Spain' 'Germany']
len:  3


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [45]:


x=pd.concat([temp_df,df.drop(columns=['Geography','Exited'])],axis=1)
y=df['Exited']
x=x.astype(dtype='float32')
y=y.astype(dtype='float32')
x.shape

(10000, 12)

# checking for imbalance dataset


In [46]:
y.value_counts()

,count
Exited,
0.0,7963
1.0,2037


#  scaling all the feature using minmaxscaler

In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_transformed=scaler.fit_transform(x)
x_transformed[:2,:]

array([[1.        , 0.        , 0.        , 0.538     , 0.        ,
        0.32432434, 0.2       , 0.        , 0.        , 1.        ,
        1.        , 0.5067349 ],
       [0.        , 0.        , 1.        , 0.51600003, 0.        ,
        0.31081083, 0.1       , 0.33403146, 0.        , 0.        ,
        1.        , 0.56270874]], dtype=float32)

#train test spliting

In [48]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=.2,random_state=42)
y_train.value_counts()

,count
Exited,
0.0,6356
1.0,1644


In [49]:
y_test.value_counts()

,count
Exited,
0.0,1607
1.0,393


# handling imbalance dataset

In [50]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)
x_train_resampled.shape

(12712, 12)

In [53]:
class mymodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(12,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,16)
    self.fc4=nn.Linear(16,4)
    self.fc5=nn.Linear(4,1)
  def forward(self,x):
    x=nn.functional.relu(self.fc1(x))
    x=nn.functional.relu(self.fc2(x))
    x=nn.functional.relu(self.fc3(x))
    x=nn.functional.relu(self.fc4(x))
    x=nn.functional.sigmoid(self.fc5(x))
    return x
model=mymodel()

In [58]:
from sklearn.metrics import precision_score,recall_score,accuracy_score
criterion =nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=.01,betas=(.8,.9))
batch_size=1024
epochs=100
for epoch in range(epochs):
  all_preds=[]
  y_true=[]
  all_preds_val=[]
  y_true_val=[]
  for i in range(0,len(x_train_resampled),batch_size):
    x_batch=torch.tensor(x_train_resampled[i:i+batch_size,:])
    y_batch=torch.tensor(np.array(y_train_resampled[i:i+batch_size]).reshape(-1,1))
    optimizer.zero_grad()
    res=model(x_batch)
    loss=criterion(res,y_batch)
    loss.backward()
    optimizer.step()

    #train eval
    all_preds.extend((res > 0.5).float())
    y_true.extend(y_batch.cpu().numpy().flatten())

  #validation eval
  model.eval()
  with torch.no_grad():
    x=torch.tensor(x_test,dtype=torch.float32)
    y=torch.tensor(np.array(y_test).reshape(-1,1))
    res_val=model(x)
    loss_val=criterion(res_val,y)
    all_preds_val.extend((res_val>.5).float())
    y_true_val.extend(y.cpu().numpy().flatten())
  model.train()

  #Accuracy
  Accuracy=accuracy_score(y_true,all_preds)
  Accuracy_val=accuracy_score(y_true_val,all_preds_val)
  #precision
  precision=precision_score(y_true,all_preds)
  precision_val=precision_score(y_true_val,all_preds_val)
  #recall
  recall=recall_score(y_true,all_preds)
  recall_val=recall_score(y_true_val,all_preds_val)


  print(f'Epoch [{epoch+1}/{ epochs}], Losss : {loss.item()}, Accuracy : {Accuracy} , precision : {precision},Loss_val :{loss_val.item()},  Accuracy_val : {Accuracy_val}, precision_val : {precision_val}')

Epoch [1/100], Losss : 0.8176300525665283, Accuracy : 0.487413467589679 , precision : 0.3586572438162544,Loss_val :0.633866548538208,  Accuracy_val : 0.792, precision_val : 0.4433497536945813
Epoch [2/100], Losss : 0.654911994934082, Accuracy : 0.6916299559471366 , precision : 0.6463238827486785,Loss_val :0.7127177715301514,  Accuracy_val : 0.2135, precision_val : 0.19989827060020346
Epoch [3/100], Losss : 0.6561169624328613, Accuracy : 0.4698709880427942 , precision : 0.4834899560306923,Loss_val :0.7090193033218384,  Accuracy_val : 0.3145, precision_val : 0.2189655172413793
Epoch [4/100], Losss : 0.6630120873451233, Accuracy : 0.46027375707992446 , precision : 0.471866295264624,Loss_val :0.7012697458267212,  Accuracy_val : 0.4525, precision_val : 0.25
Epoch [5/100], Losss : 0.658401370048523, Accuracy : 0.5096758967904342 , precision : 0.5073083778966132,Loss_val :0.6998841762542725,  Accuracy_val : 0.5015, precision_val : 0.26479750778816197
Epoch [6/100], Losss : 0.6551048755645752,